# Load, fit, and analyse the waveforms from a PMT

This script contians the required functions to load the PMT outputs from an XML file (outputs of the CAEN software) into a Pandas dataframe, then integrate the waveforms to get an integrated charge, and return the mean value

## Load the required libraries

In [1]:
!pip install -U pip
!pip install lmfit
!pip install pyarrow

In [2]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np
import pickle
import scipy.signal
import pandas as pd
import time
import traceback
from glob import glob
#from sys import argv
from os.path import splitext, exists
from lmfit.models import ConstantModel, GaussianModel
from lmfit import Model
from datetime import datetime as dt
from random import randrange
from pathlib import Path

import argparse

## Set up the constants

In [3]:
parser = argparse.ArgumentParser(description='PMT waveform analysis')
parser.add_argument('--redo_feather', action='store_true', help='if you want to regenerate this file.')
parser.add_argument('--file', '-f', type=list, help='Input file path')

_StoreAction(option_strings=['--file', '-f'], dest='file', nargs=None, const=None, default=None, type=<class 'list'>, choices=None, required=False, help='Input file path', metavar=None)

## Process the files

In [4]:
def process_files(args):
    '''
    Just store waveform in feather files.
    '''
    fnames = args.file

    for fname in fnames:
        try:
            _ = load_wforms(args,fname)
        except Exception:
            traceback.print_exc()

    return

## Analyse and integrate the waveforms

In [5]:
def load_wforms(args, fname):
    """
    Opens CAEN digitizer XML output and returns waveforms and digitiser 
    setttings.

    :param str fname: filename of input XML.
    """
    print("File: %s" % fname)
    split_fname = splitext(fname)

    feather_fname = split_fname[0]+"_wforms.feather"
    feather_clocktime_fname = split_fname[0]+'_clocktime.feather'

    print("Parsing XML...")
    # fname may be xml regardless but saves a logic gate
    try:
        tree = ET.parse(fname)
        root = tree.getroot()
        print("... done!")
    except FileNotFoundError:
        print(f"FileNotFoundError: {fname} not found.")
        print("The xml is required for digitiser info, please ensure xml files is"
            "passed (feather will automatically be used).")
        exit()

    # Get voltage range and resolution
    digi = root.find("digitizer")
    vrange_xml = digi.find("voltagerange").attrib
    # Convert strings to floats for hi and low.
    vlow = float(vrange_xml["low"])
    vhi = float(vrange_xml["hi"])
    vrange = (vlow, vhi)

    res = digi.find("resolution").attrib["bits"]
    # Convert bins to charge
    # Get voltage range from digitiser, divide by number of bins (2^number
    # of bits). 
    vbin_width = (vrange[1]-vrange[0])/(2**int(res))
    # Convert to mV
    vbin_width *= 1e3

    res = digi.find("resolution").attrib["bits"]

    # Get the trigger window
    trig_window = root.find("settings").find("window")
    trig_window = float(trig_window.attrib["size"])

    n_channels = int(digi.find("channels").attrib["value"])
    # ith wforms is the list of wforms for channel i of digi
    channels = [[] for i in range(n_channels)]
    # Store "clocktime"
    clocktime = [[] for i in range(n_channels)]

    # Check if there's a feather file to pull from
    # Skip if this is going to be regenerated
    if exists(feather_fname) and exists(feather_clocktime_fname) and (not args.redo_feather):
        print("Loading wforms from feather...")
        channels = pd.read_feather(feather_fname)
        clocktime = pd.read_feather(feather_clocktime_fname)
        # Prog written on assumption channels is list of lists of wforms
        # Transpose and put back in this form
        # TODO: Keep as pandas maybe?
        channels = channels.T.values.tolist()
        clocktime = clocktime.T.values.tolist()
        
        # Remove None-s from channels (filled when made into a df)
        for i in range(len(channels)):
            channels[i] = [x for x in channels[i] if x is not None]
            clocktime[i] = [x for x in clocktime[i] if x is not None]

        n_wforms = sum([len(wforms) for wforms in channels])
        print("... done! %i waveforms loaded." % n_wforms)

        return channels, vrange, vbin_width, trig_window, clocktime

    # Pull wforms from the xml
    print("Loading waveforms...")
    # Loops through every "event" in the XML, each with a "trace" (waveform)
    for i,child in enumerate(root.iter("event")):
        # Trace is spaced wform values. Split on spaces and convert to np array.
        # Use int as dtype to ensure numpy arithmetic.
        for wform in child.iter("trace"):
            wform_channel = int(wform.attrib["channel"])
            wform = np.array(wform.text.split()).astype(int)
            channels[wform_channel].append(wform)
            clocktime[wform_channel].append(child.attrib['clocktime'])
        
        if (i % 100) == 0:
            print(f"    {i} events loaded...\r", end="")

    # Remove unfilled (unused) channels
    channels = [channel for channel in channels if len(channel)!=0]
    clocktime = [clock for clock in clocktime if len(clock)!=0]

    print(f"... done! {i*len(channels)} waveforms loaded.")

    # Convert waveforms to dataframe, save to feather
    # Get digi info straight from the XML
    # List of lists like channels needs transposing so channels are columns
    channels_df = pd.DataFrame(channels).T
    # Feather needs string column names
    channels_df.columns = channels_df.columns.map(str)
    # Remvoe unused channels to save on space
    channels_df.dropna(how='all', axis=1, inplace=True)
    channels_df.to_feather(feather_fname)
    
    print('len(clocktime)', len(clocktime))
    clocktime_df = pd.DataFrame(clocktime).T
    clocktime_df.columns = clocktime_df.columns.map(str)
    clocktime_df.dropna(how='all', axis=1, inplace=True)
    clocktime_df.to_feather(feather_clocktime_fname)

    print(f"Saved wforms to {feather_fname}.")

    return channels, vrange, vbin_width, trig_window, clocktime

## Load the files and run the code

In [6]:
### LED measurement
files = [
    #'../../data/2024-02-22_LED/HAM_KM56213_900v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56213_950v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56213_1000v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56213_1050v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56213_1100v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56213_1150v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56213_1200v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56213_1250v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56213_1300v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_900v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_950v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_1000v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_1050v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_1100v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_1150v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_1200v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_1250v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2305-1100_1300v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_900v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_950v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_1000v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_1050v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_1100v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_1150v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_1200v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_1250v.xml',
    #'../../data/2024-02-22_LED/NNVT_PN2304-1470_1300v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_900v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_950v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_1000v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_1050v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_1100v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_1150v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_1200v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_1250v.xml',
    #'../../data/2024-02-22_LED/HAM_KM56206_1300v.xml'
    ]

### ###

### Angular dependence ###
files000 = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az000Pol00.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az000Pol15.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az000Pol30.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az000Pol40.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az000Pol50.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az000Pol60.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az000Pol70.xml'
]
files030 = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az030Pol60.xml',
]
files090 = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az090Pol00.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az090Pol15.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az090Pol30.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az090Pol40.xml',
    #'/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az090Pol45.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az090Pol50.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az090Pol60.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az090Pol70.xml'
]
files120 = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az120Pol00.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az120Pol15.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az120Pol30.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az120Pol40.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az120Pol50.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az120Pol60.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az120Pol70.xml',
]
files150 = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol00.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol15.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol30.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol40.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol50.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol60.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol70.xml',
]
files150Add = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol61.xml',
]
files180 = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az180Pol00.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az180Pol15.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az180Pol30.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az180Pol40.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az180Pol50.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az180Pol60.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az180Pol70.xml',
]
files180Add = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az180Pol51.xml',
]
files210 = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az210Pol00.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az210Pol15.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az210Pol30.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az210Pol40.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az210Pol50.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az210Pol60.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az210Pol70.xml',
]
files270 = [
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az270Pol00.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az270Pol15.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az270Pol30.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az270Pol40.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az270Pol50.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az270Pol60.xml',
    '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az270Pol70.xml',
]
files330 = [
  '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az330Pol60.xml',
]

filesValidation = [
  '/media/robertkralik/LaCie/PMTValidation/NNVT_PN23051100_1300V_Az030Pol00_Laser900.xml',
  '/media/robertkralik/LaCie/PMTValidation/NNVT_PN23051100_1300V_Az030Pol00_Laser900_2.xml',
  '/media/robertkralik/LaCie/PMTValidation/NNVT_PN23051100_1300V_Az030Pol00_Laser900_FewerEvents.xml',
  '/media/robertkralik/LaCie/PMTValidation/NNVT_PN23051100_1300V_Az030Pol00_Laser900_LaserCableMovement.xml',
  '/media/robertkralik/LaCie/PMTValidation/NNVT_PN23051100_1300V_Az030Pol00_Laser900_LightExposure.xml',
  '/media/robertkralik/LaCie/PMTValidation/NNVT_PN23051100_1300V_Az030Pol00_Laser900_MovingPMT.xml',
  '/media/robertkralik/LaCie/PMTValidation/NNVT_PN23051100_1300V_Az030Pol00_Laser900_Restart.xml'
]

#files_all = files000 + files030 + files090 + files120 + files150 + files150Add + files180 + files180Add + files210 + files270 + files330
files_single_angle = files030 + files150Add + files330

#files_all = [
#  '/media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az030Pol60_LaserInt900.xml',
#] # testing waveform fits

# Testing CoMPASS comparisons
#files_all = [ '/media/robertkralik/LaCie/TestingPMTs/Comparison_CAENScope.xml' ]

#files_all = filesValidation

files_gain_fridge = [
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_1300V.xml',
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_1250V.xml',
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_1200V.xml',
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_1150V.xml',
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_1100V.xml',
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_1050V.xml',
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_1000V.xml',
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_0950V.xml',
  '/media/robertkralik/LaCie/GainMeasurements/NNVT_PN23051100_Fridge_0900V.xml'
]

files_gain_darkbox = [
  '../../GainMeasurements/NNVT_PN2305-1100(New)_1150V.xml',
  '../../GainMeasurements/NNVT_PN2305-1100(New)_1175V.xml',
  '../../GainMeasurements/NNVT_PN2305-1100(New)_1200V.xml',
  '../../GainMeasurements/NNVT_PN2305-1100(New)_1225V.xml',
  '../../GainMeasurements/NNVT_PN2305-1100(New)_1250V.xml',
  '../../GainMeasurements/NNVT_PN2305-1100(New)_1275V.xml',
  '../../GainMeasurements/NNVT_PN2305-1100(New)_1300V.xml'
]

files_all = files_gain_darkbox

files_SPE_0 = [
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1150V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1200V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1201V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1225V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1250V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1260V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1270V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1273V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1280V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1290V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1300V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_900V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_950V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1000V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1025V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1050V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1075V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1100V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1125V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1150V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1200V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1250V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1300V.xml',
]

files_SPE = [
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1150V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1260V.xml',
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1300V.xml',
]

files_sel = [
  '../../SinglePEMeasurements/NNVT_PN2305-1100(New)_1270V.xml',
  '../../GainMeasurements/NNVT_PN2305-1100(New)_1250V.xml',
  '../../SinglePEMeasurements/HAM_KM56213(New)_1200V.xml',
]

### Dark ###
'''
files = [
    '../../from_Alex/pmt_measurements/dr/202308221609_900v.xml',
    '../../from_Alex/pmt_measurements/dr/202308221609_1000v.xml',
    '../../from_Alex/pmt_measurements/dr/202308221609_1100v.xml',
    '../../from_Alex/pmt_measurements/dr/202308221609_1200v.xml',
    '../../from_Alex/pmt_measurements/dr/202308221609_1300v.xml',
    '../../from_Alex/pmt_measurements/dr/202308221212-1600_1100v_1000v_1000v.xml',
    '../../data/2023-10-17-DR/NNVT_CH0_PN2304-1470_900v_CH1_PN2305-1100_900v.xml',
    '../../data/2023-10-17-DR/NNVT_CH0_PN2304-1470_900v_CH1_PN2305-1100_1000v.xml',
    '../../data/2023-10-17-DR/NNVT_CH0_PN2304-1470_900v_CH1_PN2305-1100_1100v.xml',
    '../../data/2023-10-17-DR/NNVT_CH0_PN2304-1470_900v_CH1_PN2305-1100_1200v.xml',
    '../../data/2023-10-17-DR/NNVT_CH0_PN2304-1470_900v_CH1_PN2305-1100_1300v.xml'
]
'''

files_dr = [
  '../../DarkRate/Ch11-NNVT_PN2305-1100(New)_1270V_Ch15-HAM_KM56213_1050V_AfterWeekend.xml',
  '../../DarkRate/Ch11-NNVT_PN2305-1100(New)_1273V_Ch15-HAM_KM56213_1050V_After5Min.xml',
  '../../DarkRate/Ch11-NNVT_PN2305-1100(New)_1273V_Ch15-HAM_KM56213_1050V_ShorterPulse.xml',
  '../../DarkRate/Ch11_NNVT_PN2305-1100_1270V_Ch15_HAM_KM56213_1060V_Monday19082024.xml',
  '../../DarkRate/Ch11_NNVT_PN2305-1100_1270V_Ch15_HAM_KM56213_1060V_Monday19082024_Faster.xml',
]

#files = [
    #'../../data/2024-02-05_DR_LONG/CH0_KM56206_1076v_CH1_KM56213_983v_CH2_PN2304-1470_1014v_CH3_PN2305-1100_1195v-1.xml',
    #'../../data/2024-02-05_DR_LONG/CH0_KM56206_1076v_CH1_KM56213_983v_CH2_PN2304-1470_1014v_CH3_PN2305-1100_1195v-2.xml',
    #'../../data/2024-02-05_DR_LONG/CH0_KM56206_1076v_CH1_KM56213_983v_CH2_PN2304-1470_1014v_CH3_PN2305-1100_1195v-3.xml',
    #'../../data/2024-02-05_DR_LONG/CH0_KM56206_1076v_CH1_KM56213_983v_CH2_PN2304-1470_1014v_CH3_PN2305-1100_1195v-4.xml',
    #'../../data/2024-02-05_DR_LONG/CH0_KM56206_1076v_CH1_KM56213_983v_CH2_PN2304-1470_1014v_CH3_PN2305-1100_1195v-5.xml',
    #'../../data/2024-02-05_DR_LONG/CH0_KM56206_1076v_CH1_KM56213_983v_CH2_PN2304-1470_1014v_CH3_PN2305-1100_1195v-6.xml'
#]
#files = [
#    '../../data/2024-02-15_DR/CH0_PN2304-1470_1025v_CH1_H11901P-04_0.9v_wXeLight.xml'
#]
### ###

### Gain=5e6 ###

#files = [
#    '../../data/2024-02-22_LED/HAM_KM56213_1058v.xml',
#    '../../data/2024-02-22_LED/HAM_KM56206_1122v.xml',
#    '../../data/2024-02-22_LED/NNVT_PN2304-1470_1066v.xml',
#    '../../data/2024-02-22_LED/NNVT_PN2305-1100_1345v.xml'
#]
#files = [
#    '../../data/2024-02-22_LED/NNVT_PN2305-1100_1250v.xml',
#    '../../data/2024-02-22_LED/NNVT_PN2305-1100_1300v.xml'
#]
### ###


In [7]:
options = ['-f', files_single_angle] # Just store waveform in feather files (for QE test, etc.)

args = parser.parse_args(args=options)

fnames = args.file
process_files(args)


File: /media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az030Pol60.xml
Parsing XML...
FileNotFoundError: /media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az030Pol60.xml not found.
The xml is required for digitiser info, please ensure xml files ispassed (feather will automatically be used).
File: /media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol61.xml
Parsing XML...
FileNotFoundError: /media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az150Pol61.xml not found.
The xml is required for digitiser info, please ensure xml files ispassed (feather will automatically be used).
File: /media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az330Pol60.xml
Parsing XML...
FileNotFoundError: /media/robertkralik/LaCie/PMTMeasurements/NNVT_PN23051100_1300V_Az330Pol60.xml not found.
The xml is required for digitiser info, please ensure xml files ispassed (feather will automatically be used).


Traceback (most recent call last):
  File "/tmp/ipykernel_207503/2736223388.py", line 9, in process_files
    _ = load_wforms(args,fname)
        ^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_207503/259393641.py", line 27, in load_wforms
    digi = root.find("digitizer")
           ^^^^
UnboundLocalError: cannot access local variable 'root' where it is not associated with a value
Traceback (most recent call last):
  File "/tmp/ipykernel_207503/2736223388.py", line 9, in process_files
    _ = load_wforms(args,fname)
        ^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_207503/259393641.py", line 27, in load_wforms
    digi = root.find("digitizer")
           ^^^^
UnboundLocalError: cannot access local variable 'root' where it is not associated with a value
Traceback (most recent call last):
  File "/tmp/ipykernel_207503/2736223388.py", line 9, in process_files
    _ = load_wforms(args,fname)
        ^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_207503/259393641.py", line 27, in load_

: 